In [1]:
# -------------- #

import pandas as pd
import os
import datetime
import math

import load_data
from util_log import logger
# from featureEng import (
# )
# from models import grid_search
from config import (
    FEATUER_DIR,
    LAG_COLS,
    LAG_LENGTH
)

import run_model

# valid period. '2017-07-31' to '2017-08-16'
SPLIT_DATE = datetime.date(2017,7, 31)

def lagging(df,
            lag_len=50,
            lag_cols=['unit_sales']
            ):
    """ Generate lagging features.
    Args:
        df: a DataFrame, contains all columns in cols_lag
        len_lag: Int, the length of lagging features
        cols_lag: List of String, the columns needs to generate lagging features 

    Returns:
        df: a DataFrame with lagging features, for example 'Sales_lag_1'.
    """

    # lag
    for lag in range(1, lag_len):
        _df_shift = df[lag_cols].shift(periods=lag)
        for col in lag_cols:
            df[col + '_lag_' + str(lag)] = _df_shift[col]
    return df

/Users/liuyu/Desktop/g-forecast/config.py:8: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config = yaml.load(f)


-------------------------------------------------------------
configurations: 
{
  "storeId": null,
  "influence_sheet": "",
  "influence_min": 0,
  "influence_max": 0,
  "predict_start_date": "2019-06-23",
  "predict_length": 35,
  "lag_length": 50,
  "lag_cols": [
    "Sales"
  ],
  "sliding_window_length": 50,
  "sliding_cols": [
    "Sales"
  ],
  "model_list": [
    "LGBMRegressor"
  ],
  "peak_period": [
    "2017-12-08",
    "2018-02-08",
    "2018-12-10",
    "2019-02-10"
  ]
}
-------------------------------------------------------------


In [2]:
logger.debug('Get raw data')
train_df, test_df = load_data.main()
train_df = train_df[['date', 'store_nbr', 'item_nbr', 'unit_sales']]
train_df = train_df[train_df['date']>=datetime.date(2016, 1, 1)]
train_df = train_df.dropna()


Get raw data
/Users/liuyu/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3296: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/liuyu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  after removing the cwd from sys.path.


In [3]:
storeId_list = train_df['store_nbr'].unique()
# logger.debug('Filling missing value, lag, sliding window')
for storeId in storeId_list:
    store_frame = train_df[train_df['store_nbr'] == storeId]
    item_list = store_frame.item_nbr.unique()
    logger.debug(storeId)
    count = 0

    for itemId in item_list:
        item_frame = store_frame[store_frame['item_nbr']==itemId]
        item_frame = item_frame.sort_values(by=['date'])
        item_frame = item_frame.reset_index(drop=True)
        item_frame = lagging(
            item_frame,
            lag_len=LAG_LENGTH, 
            lag_cols=LAG_COLS)

        if count == 0:
            df = item_frame
        else:
            df = pd.concat([df, item_frame])
        count += 1

    df.to_csv(FEATUER_DIR+'{}_lag_sales.csv'.format(storeId), index=False)
    del df

25
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
53
54
52
